In [12]:
import numpy as np
import matplotlib as mpl
from matplotlib import figure, animation
import scipy as sci
mpl.rcParams.update({
    'font.family': 'serif',
    'text.usetex': True,
    'axes.labelsize': 'large',
    'figure.dpi': 100,
})

import numba
import os
import glob
import subprocess
import re

In [271]:
def fig_io(fig, savefig, viewfig):
    '''
    Save figure and optionaly open it in an image viewer.
    '''
    if (not savefig) and viewfig:
        raise ValueError(
            'Setting `viewfig=True` is only'
            'valid when `savefig` is not `None`.'
        )
        
    if savefig:
        lowername = str.lower(savefig)
        is_img = (
            str.endswith(lowername, '.png')
            or str.endswith(lowername, '.jpg')
            or str.endswith(lowername, '.webp')
        )
        is_pdf = str.endswith(lowername, '.pdf')
        
        if not (is_img or is_pdf):
            raise ValueError(
                'Name for saved file `savefig` must have'
                'extension pdf, png, jpg or webp, '
                f'`{savefig}` is not a valid `savefig` value.'
            )
        
        # Save figure.
        absname = os.path.abspath(savefig)
        fig.savefig(absname, dpi=300)
            
        if viewfig:
            cmd = 'qview' if is_img else 'zathura'
            subprocess.Popen([cmd, absname])

In [418]:
# Import all measurements from the `./measurements` directory. Each measurement file
# is read into a numpy array (column-first) and added to the `meas_dict` (keys are
# filenames with extentions removed).
meas_dict = {}
suffix, sep = 'csv', ','
for fname in glob.glob(f'./measurements/*.{suffix}'):
    with open(fname) as file:
        meas = [[float(el) for el in row.rstrip().split(sep)]
                for row in file.readlines() if not (str.startswith(row, '#') or str.isspace(row))]
        meas_name, _ = str.rsplit(os.path.basename(fname), '.', maxsplit=1)
        meas_dict[meas_name] = np.array(meas).T

In [276]:
from scipy.optimize import curve_fit
from matplotlib.ticker import AutoMinorLocator

@numba.njit
def gauss(x, μ, σ, C):
    return (
        C/(σ*np.sqrt(2*np.pi))
        * np.exp(-1/2 * (x-μ)**2 / σ**2)
    )

@numba.njit
def subsum(arr, factor):
    N = len(arr) # Only for 1D arrays
    N_new = N//factor
    arr_new = np.zeros(N_new)
    for i in range(N_new):
        start = (N*i) // N_new
        end = (N * (i+1)) // N_new
        arr_new[i] = np.sum(arr[start:end])
    return arr_new

def fig_ionization(
    meas_name,
    subsample_factor=8,
    plot_background_spectrum=False,
    savefig=None,
    viewfig=False,
):
    fig = mpl.figure.Figure([6, 4.5])
    ax = fig.subplots(1, 1)
    
    # --- Unpack measurements ---
    
    def unpack_spectrum(meas_name):
        meas = meas_dict[meas_name][0]
        livetime, realtime = meas[:2] # Analyser pulses
        A = meas[2:] / livetime
        A = subsum(A, subsample_factor) # Subsample
        #print(f'{meas_name}: livetime = {livetime}')
        return A # Enery window index and radioactivity
    
    A_back = unpack_spectrum('background')
    A = unpack_spectrum(meas_name) - A_back # Subtract background
        
    # --- Calibrate energy scale ---
    
    def gaussian_peak_index(A, a, b, p0):
        index = np.arange(len(A))
        # Fit Gaussian
        a, b = a//subsample_factor, b//subsample_factor
        p0 = 1/subsample_factor * np.array(p0)
        par, cov = curve_fit(gauss, index[a:b], A[a:b], p0)
        μ, σ, C = par
        #ax.plot(index[a:b], A[a:b], color='r')
        #ax.plot(index, gauss(index, *par), color='r', lw=0.5, ls='--')
        return μ, σ, C
    
    A_na22 = unpack_spectrum('na22') - A_back # Subtract background
    E1_index, σ1, C1 = gaussian_peak_index(A_na22, 8*150, 8*180, p0=[8*165, 8*5, 20])
    E2_index, σ2, C2 = gaussian_peak_index(A_na22, 8*380, 8*410, p0=[8*395, 8*10, 5])
    
    index = np.arange(len(A_na22))
    x = (index - E1_index) / (E2_index - E1_index)
    E1, E2 = 511e3, 1277e3 # eV
    E = (E2 - E1) * x + E1
    # Strech Gaussian.
    k = (E2 - E1) / (E2_index - E1_index)
    μ_E1, σ_E1, C1 = E[int(E1_index)], k*σ1, k*C1
    μ_E2, σ_E2, C2 = E[int(E2_index)], k*σ2, k*C2
    
    # --- Normalize to density ---
    
    E_span = np.max(E) - np.min(E)
    dE = E_span / len(A)
    
    A_total = np.sum(A)
    A = 1/dE * A
    C1 = 1/dE * C1 # Scale Gaussian
    #
    A_back_total = np.sum(A_back)
    A_back = 1/dE * A_back
    C2 = 1/dE * C2 # Scale Gaussian
    
    print((C1, C2))
    
    # --- Plot spectra ---
    
    def label_from_meas_name(meas_name):
        elname, elnum = re.match(r'(\D+)(\d+)$', meas_name).groups()
        return f'^{{{elnum}}}\\mathrm{{{str.capitalize(elname)}}}'
    
    plt, = ax.plot(
        1e-3 * E, 1e3 * A,
        lw=0.8,
        label=r'$' f'{label_from_meas_name(meas_name)}' '$ (brez ozadja)'
    )
    ax.fill_between(
        1e-3 * E, 1e3 * A,
        color=plt.get_color(), alpha=0.2,
        label=f'skupna aktivnost ${A_total:.0f}\\,\\mathrm{{Bq}}$'
    )
    
    ax.plot(1e-3 * E, 1e3 * gauss(E, μ_E1, σ_E1, C1), color='r', lw=0.8, ls='--')
    ax.plot(1e-3 * E, 1e3 * gauss(E, μ_E2, σ_E2, C2), color='r', lw=0.8, ls='--')
    
    if plot_background_spectrum:
        plt, = ax.plot(
            1e-3 * E, 1e3 * A_back,
            lw=0.8,
            label='ozadje'
        )
        ax.fill_between(
            1e-3 * E, 1e3 * A_back,
            color=plt.get_color(), alpha=0.2,
            label=f'skupna aktivnost ${A_back_total:.0f}\\,\\mathrm{{Bq}}$'
        )
    
    # --- Set ---
    ax.set(
        title=r'$\gamma$ spekter za $' f'{label_from_meas_name(meas_name)}' r'$',
        xlabel=r'$E\,[\mathrm{keV}]$',
        ylabel=r'gostota $\frac{\mathrm{d}A}{\mathrm{d}E}\,[\mathrm{Bq}/\mathrm{keV}]$',
        xlim=(0,2000),
        ylim=(0,10),
        #yscale='log',
    )
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    #fig.subplots_adjust(top=0.8)
    ax.legend(frameon=False)
    
    # --- Figure I/O ---
    fig_io(fig, savefig, viewfig)
        
fig_ionization('na22', plot_background_spectrum=True, savefig='na22-spectrum.pdf', viewfig=False)

(422.29392991276114, 68.99589965040272)


In [505]:
def comptonpeak(E_γ):
    return E_γ / (1 + 2 * E_γ/511e3)

def fig_ionization_lost(
    meas_name,
    savefig=None,
    viewfig=False,
):
    fig = mpl.figure.Figure([6, 4.5])
    ax = fig.subplots(1, 1)
    
    # --- Unpack measurements ---
    
    def unpack_spectrum(meas_name):
        meas = meas_dict[meas_name][0]
        livetime, realtime = meas[:2] # Analyser pulses
        A = meas[2:] / livetime
        print(f'{meas_name}: livetime = {livetime}')
        return A # Enery window index and radioactivity
    
    A_back = unpack_spectrum('background-lost')
    A = unpack_spectrum(meas_name) - A_back # Subtract background
    
    # --- Calibrate energy scale ---
    
    def gaussian_peak_index(A, a, b, p0):
        index = np.arange(len(A))
        # Fit Gaussian
        par, cov = curve_fit(gauss, index[a:b], A[a:b], p0)
        μ, σ, A = par
        _, _, ΔA = np.sqrt(par)
        return μ, σ, (A, ΔA)
    
    A_na22 = unpack_spectrum('na22-lost') - A_back # Subtract background
    E1_index, σ_E1_index, (A1, ΔA1) = gaussian_peak_index(A_na22, 130, 155, p0=[140, 6, 500])
    E2_index, σ_E2_index, (A2, ΔA2) = gaussian_peak_index(A_na22, 320, 360, p0=[340, 12, 100])
    
    index = np.arange(len(A_na22))
    
    x = (index - E1_index) / (E2_index - E1_index)
    E1, E2 = 511e3, 1277e3 # eV
    E = (E2 - E1) * x + E1
    
    # --- Normalize to density ---
    
    E_span = np.max(E) - np.min(E)
    dE = E_span / len(A)
    
    A_total = np.sum(A)
    A = 1/dE * A
    
    A_back_total = np.sum(A_back)
    A_back = 1/dE * A_back
    
    # --- Plot ---
        
    def label_from_meas_name(meas_name):
        elname, elnum = re.match(r'(\D+)(\d+)-lost$', meas_name).groups()
        return f'^{{{elnum}}}\\mathrm{{{str.capitalize(elname)}}}'
    
    plt, = ax.plot(
        1e-3 * E, 1e3 * A,
        lw=0.8,
        label=r'spekter $' f'{label_from_meas_name(meas_name)}' '$ (brez ozadja)'
    )
    ax.fill_between(
        1e-3 * E, 1e3 * A,
        color=plt.get_color(), alpha=0.2,
        label=f'skupna aktivnost ${A_total:.0f}\\,\\mathrm{{Bq}}$'
    )
    
    if meas_name == 'na22-lost':
        # Background
        plt, = ax.plot(
            1e-3 * E, 1e3 * A_back,
            lw=0.8, color='orange',
            label='ozadje'
        )
        ax.fill_between(
            1e-3 * E, 1e3 * A_back,
            color=plt.get_color(), alpha=0.2,
            label=f'skupna aktivnost ${A_back_total:.0f}\\,\\mathrm{{Bq}}$'
        )
        # Gaussian peaks
        σ_E1 = (E2 - E1) / (E2_index - E1_index) * σ_E1_index
        σ_E2 = (E2 - E1) / (E2_index - E1_index) * σ_E2_index
        ax.plot(
            1e-3 * E, 1e3 * gauss(E, E1, σ_E1, A1),
            color='r', lw=0.8, ls='--', zorder=10,
            label='prilagojena Gaussovka'
        )
        ax.plot(
            1e-3 * E, 1e3 * gauss(E, E2, σ_E2, A2),
            color='r', lw=0.8, ls='--', zorder=10
        )
        R1 = 2.355 * σ_E1 / E1
        ax.text(
            E1/2000e3, 0.85,
            (
                f'$({1e-3 * E1:.0f} \\pm {1e-3 * σ_E1:.0f})\\,\\mathrm{{keV}},$\n'
                f'$({A1:.0f} \\pm {ΔA1:.0f})\\,\\mathrm{{Bq}},$\n'
                f'$\\; R = {R1:.3f}$'
            ),
            verticalalignment='center', horizontalalignment='center',
            transform=ax.transAxes, fontsize=12
        )
        R2 = 2.355 * σ_E2 / E2
        ax.text(
            E2/2000e3, 0.2,
            (
                f'$({1e-3 * E2:.0f} \\pm {1e-3 * σ_E2:.0f})\\,\\mathrm{{keV}}$,\n'
                f'$({A2:.0f} \\pm {ΔA2:.0f})\\,\\mathrm{{Bq}},$\n'
                f'$\\; R = {R2:.3f}$'
            ),
            verticalalignment='center', horizontalalignment='center',
            transform=ax.transAxes, fontsize=12
        )
        # Predicted Compton backscattering peak
        ax.plot(
            [1e-3 * comptonpeak(511e3)]*2, [0, 1e10],
            color='purple', lw=0.8, ls='--', zorder=0,
            label='predvideni Comptonski vrh'
        )
        ax.plot(
            [1e-3 * comptonpeak(1277e3)]*2, [0, 1e10],
            color='purple', lw=0.8, ls='--', zorder=0
        )
        # SCA
        SCA_U, SCA_N = meas_dict['sca']
        SCA_E = (E2 - E1)/(3.75 - 1.6) * (SCA_U - 1.6) + E1
        SCA_dE = (E2 - E1)/(3.75 - 1.6) * 0.3
        SCA_A = SCA_N / (10.0 * SCA_dE)
        ax.plot(
            1e-3 * SCA_E, 1e3 * SCA_A,
            color='g', markersize=3, ls='', marker='^', zorder=10,
            label='meritve z enokanalnim\nanalizatorjem'
        )
        y_max=14
    elif meas_name == 'co60-lost':
        a, b = 290, 330
        par1, cov1 = curve_fit(gauss, E[a:b], A[a:b], p0=[1150e3, 50e3, 5e-2])
        a, b = 330, 380
        par2, cov2 = curve_fit(gauss, E[a:b], A[a:b], p0=[1450e3, 50e3, 5e-2])
        ax.plot(
            1e-3 * E, 1e3 * gauss(E, *par1),
            color='r', lw=0.8, ls='--', zorder=10,
            label='prilagojena Gaussovka'
        )
        ax.plot(
            1e-3 * E, 1e3 * gauss(E, *par2),
            color='r', lw=0.8, ls='--', zorder=10
        )
        R1 = 2.355 * par1[1] / par1[0]
        ax.text(
            par1[0]/2000e3, 0.35,
            (
                f'$({1e-3 * par1[0]:.0f} \\pm {1e-3 * par1[1]:.0f})\\,\\mathrm{{keV}}$,\n'
                f'$({par1[2]:.0f} \\pm {np.sqrt(cov1[2,2]):.0f})\\,\\mathrm{{Bq}},$\n'
                f'$\\; R = {R1:.3f}$'
            ),
            verticalalignment='bottom', horizontalalignment='right',
            transform=ax.transAxes, fontsize=12
        )
        R2 = 2.355 * par2[1] / par2[0]
        ax.text(
            par2[0]/2000e3, 0.3,
            (
                f'$({1e-3 * par2[0]:.0f} \\pm {1e-3 * par2[1]:.0f})\\,\\mathrm{{keV}}$,\n'
                f'$({par2[2]:.0f} \\pm {np.sqrt(cov2[2,2]):.0f})\\,\\mathrm{{Bq}},$\n'
                f'$\\; R = {R2:.3f}$'
            ),
            verticalalignment='bottom', horizontalalignment='left',
            transform=ax.transAxes, fontsize=12
        )
        # Predicted Compton backscattering peak
        ax.plot(
            [1e-3 * comptonpeak(0.5 * (par1[0] + par2[0]))]*2, [0, 1e10],
            color='purple', lw=0.8, ls='--', zorder=0,
            label='predvideni Comptonski vrh'
        )
        y_max=2
    elif meas_name == 'cs137-lost':
        a, b = 160, 200
        par1, cov1 = curve_fit(gauss, E[a:b], A[a:b], p0=[700e3, 50e3, 15e-2])
        R1 = 2.355 * par1[1] / par1[0]
        ax.plot(
            1e-3 * E, 1e3 * gauss(E, *par1),
            color='r', lw=0.8, ls='--', zorder=10,
            label='prilagojena Gaussovka'
        )
        ax.text(
            par1[0]/2000e3, 0.8,
            (
                f'$({1e-3 * par1[0]:.0f} \\pm {1e-3 * par1[1]:.0f})\\,\\mathrm{{keV}}$,\n'
                f'$({par1[2]:.0f} \\pm {np.sqrt(cov1[2,2]):.0f})\\,\\mathrm{{Bq}},$\n'
                f'$\\; R = {R1:.3f}$'
            ),
            verticalalignment='center', horizontalalignment='center',
            transform=ax.transAxes, fontsize=12
        )
        # Predicted Compton backscattering peak
        ax.plot(
            [1e-3 * comptonpeak(par1[0])]*2, [0, 1e10],
            color='purple', lw=0.8, ls='--', zorder=0,
            label='predvideni Comptonski vrh'
        )
        y_max=6
    
    # --- Set ---
    ax.set(
        title=r'$\gamma$ spekter za $' f'{label_from_meas_name(meas_name)}' r'$',
        xlabel=r'$E\,[\mathrm{keV}]$',
        ylabel=r'gostota $\frac{\mathrm{d}A}{\mathrm{d}E}\,[\mathrm{Bq}/\mathrm{keV}]$',
        xlim=(0,800 if meas_name == 'cs137' else 2000),
        ylim=(0,y_max),
        #yscale='log',
    )
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    #fig.subplots_adjust(top=0.8)
    ax.legend(loc='upper right', frameon=False)
    
    # --- Figure I/O ---
    fig_io(fig, savefig, viewfig)
    
fig_ionization_lost('na22-lost', savefig='na22-lost-spectrum.pdf', viewfig=False)
fig_ionization_lost('co60-lost', savefig='co60-lost-spectrum.pdf', viewfig=False)
fig_ionization_lost('cs137-lost', savefig='cs137-lost-spectrum.pdf', viewfig=False)

background-lost: livetime = 307.053333
na22-lost: livetime = 63.79
na22-lost: livetime = 63.79
background-lost: livetime = 307.053333
co60-lost: livetime = 200.306667
na22-lost: livetime = 63.79
background-lost: livetime = 307.053333
cs137-lost: livetime = 128.238
na22-lost: livetime = 63.79


In [402]:
A_peak, ΔA_peak = 195, 2
A0 = 9250
t2 = 30.07 # In years
t, Δt = 9, 3/12 # In years
A = A0/2 * 2**(-t/t2)

η = A_peak/A
Δη = np.sqrt( (1/A0 * 2**(-t/t2) * ΔA_peak)**2 + (A_peak/(A0*t2) * 2**(-t/t2) * Δt) )
η, Δη

(0.05188258933303369, 0.011935686662757933)